In [1]:
import os

os.environ["OPENAI_API_KEY"] = "sk-xxx"

In [4]:
from pprint import pprint
from openai import OpenAI, AsyncOpenAI, AsyncStream

client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
async_client = AsyncOpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

In [7]:
# messages = [
#     {
#         "role": "user",
#         "content": "日本で一番高い山は何?",
#     }
# ]
# completion = client.chat.completions.create(
#     model="gpt-3.5-turbo",
#     messages=messages,
#     temperature=0.0,
# )
# pprint(completion)

ChatCompletion(id='chatcmpl-9HiRbY4eXx8sF3SlMyPZwH0OZAPAb', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='日本で一番高い山は富士山です。標高は3,776メートルです。', role='assistant', function_call=None, tool_calls=None))], created=1714009447, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint='fp_c2295e73ad', usage=CompletionUsage(completion_tokens=29, prompt_tokens=18, total_tokens=47))


In [8]:
# messages = [
#     {
#         "role": "user",
#         "content": "日本で一番高い山は何?",
#     },
#     {
#         "role": "assistant",
#         "content": "日本で一番高い山は富士山です。標高は3,776メートルです。",
#     },
#     {
#         "role": "user",
#         "content": "一つ前の発言をひらがなに変換してください。",
#     },
# ]
# completion = client.chat.completions.create(
#     model="gpt-3.5-turbo",
#     messages=messages,
#     temperature=0.0,
# )
# pprint(completion)

ChatCompletion(id='chatcmpl-9HiUcoa1AkLTIUWQdnSMtzshW4lF3', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='にほんでいちばんたかいやまはふじさんです。ひょうこうは3,776メートルです。', role='assistant', function_call=None, tool_calls=None))], created=1714009634, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint='fp_c2295e73ad', usage=CompletionUsage(completion_tokens=37, prompt_tokens=76, total_tokens=113))


In [9]:
# messages = [
#     {
#         "role": "user",
#         "content": "数字を1から10まで読み上げてください。",
#     }
# ]
# completion = client.chat.completions.create(
#     model="gpt-3.5-turbo",
#     messages=messages,
#     temperature=0.0,
#     max_tokens=1,
# )
# pprint(completion)

ChatCompletion(id='chatcmpl-9HiWatVBmFUJy7pCBohFTTKEGooAM', choices=[Choice(finish_reason='length', index=0, logprobs=None, message=ChatCompletionMessage(content='1', role='assistant', function_call=None, tool_calls=None))], created=1714009756, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint='fp_c2295e73ad', usage=CompletionUsage(completion_tokens=1, prompt_tokens=22, total_tokens=23))


In [10]:
# messages = [
#     {
#         "role": "user",
#         "content": "数字を1から10まで読み上げてください。",
#     }
# ]
# completion = client.chat.completions.create(
#     model="gpt-3.5-turbo",
#     messages=messages,
#     temperature=0.0,
#     stop="5",
# )
# pprint(completion)

ChatCompletion(id='chatcmpl-9HiX3m55O7jHcQZPjs0sOkNGaPKJT', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='1, 2, 3, 4, ', role='assistant', function_call=None, tool_calls=None))], created=1714009785, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint='fp_c2295e73ad', usage=CompletionUsage(completion_tokens=12, prompt_tokens=22, total_tokens=34))


# 9.3.2 効率的なリクエスト送信

- openai API v1.0.0化に伴い、OpenAIErrorがなくなっている
  - https://github.com/openai/openai-python/discussions/742#discussioncomment-8138046
- retryも入っている
  - https://github.com/openai/openai-python?tab=readme-ov-file#retries
- ソースコード的には、
  - `retry_on_error`
    - リトライ機構はOpenAPIが用意したものを使う
      - リトライ回数は5回に
      - この関数の定義はいらなそう
  - `_async_batch_run_chatgpt`
    - Async client を使う
      - https://github.com/openai/openai-python/discussions/742

In [5]:
import asyncio

In [15]:
async def _async_batch_run_chatgpt(
    messages_list: list[list[dict[str, str]]],
    temperature: float,
    max_tokens: int | None,
    stop: str | list[str] | None,
) -> list[str]:
    """OpenAI APIに並行してリクエストを送る"""
    tasks = [
        async_client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=ms,
            temperature=temperature,
            max_tokens=max_tokens,
            stop=stop,
        ) for ms in messages_list
    ]
    # tasks内の非同期処理を実行し結果を収集
    completions = await asyncio.gather(*tasks)
    return [
        c.choices[0].message.content for c in completions
    ]

In [28]:
def batch_run_chatgpt(
    messages_list: list[list[dict[str, str]]],
    temperature: float = 0.0,
    max_tokens: int | None = None,
    stop: str | list[str] | None = None,
) -> list[str]:
    return asyncio.run(
        _async_batch_run_chatgpt(
            messages_list,
            temperature,
            max_tokens,
            stop
        )
    )

In [8]:
import nest_asyncio

nest_asyncio.apply()

# 9.3.3 クイズ用プロンプト

In [9]:
from abc import ABC, abstractmethod


class PromptMaker(ABC):
    @abstractmethod
    def run(self, questions: list[str]) -> list[str]:
        pass

In [10]:
class SimplePromptMaker(PromptMaker):
    def run(self, questions):
        return [
            "あなたには今からクイズに答えてもらいます。"
            "問題を与えますので、その解答のみを簡潔に出力してください。\n"
            f"問題：{q}\n"
            "解答："
            for q in questions
        ]

In [11]:
questions = [
    "日本で一番高い山は何?",
    "日本で一番長い川は何?",
    "日本で一番面積の大きい都道府県はどこ?",
    "日本で一番人口の多い都道府県はどこ?",
]
simple_prompt_maker = SimplePromptMaker()
print(simple_prompt_maker.run(questions))

['あなたには今からクイズに答えてもらいます。問題を与えますので、その解答のみを簡潔に出力してください。\n問題：日本で一番高い山は何?\n解答：', 'あなたには今からクイズに答えてもらいます。問題を与えますので、その解答のみを簡潔に出力してください。\n問題：日本で一番長い川は何?\n解答：', 'あなたには今からクイズに答えてもらいます。問題を与えますので、その解答のみを簡潔に出力してください。\n問題：日本で一番面積の大きい都道府県はどこ?\n解答：', 'あなたには今からクイズに答えてもらいます。問題を与えますので、その解答のみを簡潔に出力してください。\n問題：日本で一番人口の多い都道府県はどこ?\n解答：']


In [16]:
answers = batch_run_chatgpt(
    messages_list=[[{"role": "user", "content": p}] for p in simple_prompt_maker.run(questions)],
    temperature=0.0,
)
print(answers)

['富士山', '信濃川', '北海道', '東京都']


# 9.3.4 API料金見積もり

In [17]:
import tiktoken

In [18]:
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
encoding.encode("日本で一番高い山は何?")

[9080, 22656, 16556, 15120, 87217, 45736, 16995, 58911, 15682, 99849, 30]

In [19]:
from datasets import load_dataset

In [20]:
quiz_dataset = load_dataset("llm-book/aio", split="validation")

Generating train split:   0%|          | 0/22335 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [22]:
max_answer_length = 0
for answers in quiz_dataset["answers"]:
    for answer in answers:
        answer_length = len(encoding.encode(answer))
        max_answer_length = max(max_answer_length, answer_length)
print(max_answer_length)

33


- コストの見積もりは省略

# 9.3.5 クイズデータセットによる評価

In [25]:
from datasets import Dataset
from tqdm import tqdm


def get_chatgpt_outputs_for_quiz(
    quiz_prompt_maker: PromptMaker,
    quiz_dataset: Dataset,
    batch_size: int,
) -> list[str]:
    """クイズ用プロンプトを使用した際のデータセットの核問題に対するモデルの解答を集める"""
    output_answers: list[str] = []
    with tqdm(total=len(quiz_dataset)) as pbar:
        for batch in quiz_dataset.iter(batch_size=batch_size):
            prompts = quiz_prompt_maker.run(batch["question"])
            inputs = [
                [{
                    "role": "user",
                    "content": p
                }] for p in prompts
            ]
            # APIリクエスト送信
            answers = batch_run_chatgpt(inputs)

            for question, answer in zip(batch["question"], answers):
                print(f"問題：{question}")
                print(f"解答：{answer}")
                print()

            output_answers += answers
            pbar.update(len(answers))
    return output_answers

In [26]:
def calculate_quiz_accuracy(
    output_answers: list[str],
    correct_answers_list: list[list[str]],
) -> float:
    num_correct = 0
    for output_answer, answers in zip(output_answers, correct_answers_list):
        # 解答例の少なくとも1つが、モデルの出力に含まれていれば正解とみなす
        num_correct += int(any(a in output_answer for a in answers))
    return num_correct / len(output_answers)

In [29]:
output_answers = get_chatgpt_outputs_for_quiz(
    simple_prompt_maker,
    quiz_dataset,
    batch_size=4,
)
accuracy = calculate_quiz_accuracy(
    output_answers,
    [item["answers"] for item in quiz_dataset],
)
print(f"正解率：{accuracy * 100}")

  0%|          | 4/1000 [00:01<06:35,  2.52it/s]

問題：映画『ウエスト・サイド物語』に登場する2つの少年グループといえば、シャーク団と何団?
解答：ジェット団

問題：氷った海に穴を開けて漁をすることから、漢字で「氷の下の魚」と書くタラ科の魚は何?
解答：鱈

問題：アメリカとロシア間で結ばれた「戦略兵器削減条約」のことをアルファベット5文字の略称で何という?
解答：START

問題：リンゴが木から落ちる様子を見て「万有引力」を発見したといわれる、17世紀のイギリスの科学者は誰?
解答：アイザック・ニュートン



  1%|          | 8/1000 [00:02<04:40,  3.53it/s]

問題：聖武天皇の時代に栄えた、東大寺正倉院や唐招提寺金堂など、中国・唐の影響を強く受けた貴族文化を何という?
解答：奈良時代

問題：アメリカ・メジャーリーグの2つのリーグといえば、ナショナルリーグと何リーグ?
解答：アメリカンリーグ

問題：池坊、草月流、小原流といえば、どんな分野の三大流派?
解答：華道

問題：英語で「取り巻く」という意味がある、前後左右から音が聞こえる立体音響を何という?
解答：Surround sound



  1%|          | 12/1000 [00:03<04:45,  3.47it/s]

...

問題：「冬の大三角」を形成する3つの星は、おおいぬ座のシリウス、こいぬ座のプロキオンとオリオン座の何?
解答：ベテルギウス

問題：江戸時代末期・大坂に洗心堂という私塾を開いた儒学者で、1837年に江戸幕府に対して反乱を起こすも鎮圧され自害した人物は誰?
解答：関孝和

問題：『ブエノスアイレス午前零時』で芥川賞を受賞したのは藤沢周ですが、『暗殺の年輪』で直木賞を受賞したのは誰?
解答：村上春樹

問題：手紙やハガキを送った際に、料金を支払った証拠として郵便局で押される、日付入りのスタンプのことを何という?
解答：消印



100%|█████████▉| 996/1000 [05:14<00:01,  3.85it/s]

問題：NBAや格闘技などでみられる、選手が対戦相手に対して挑発的な言葉を発して心理的な揺さぶりをかける戦術を何という?
解答：トラッシュトーク

問題：いい加減で役立たずな人のことを、落語によく登場する間抜けな人物にたとえて何という?
解答：「ボケ」

問題：パーソン、パンサー、ジャクソンなどの種類がいる、周囲の環境に合わせて体の色を変えることでおなじみの動物といえば何?
解答：カメレオン

問題：小説『古都』や『伊豆の踊子』などの作品を書き、1968年にノーベル文学賞を受賞した作家は誰?
解答：川端康成



100%|██████████| 1000/1000 [05:15<00:00,  3.17it/s]

問題：いわゆる「古典の三大随筆」と呼ばれる3つの作品といえば、『枕草子』、『方丈記』と何?
解答：『徒然草』

問題：正式名称を「新幹線電気軌道総合試験車」という、新幹線の線路や架線の状態を調査するための車両を車体の色から何という?
解答：黄色

問題：明治11年、東京・紀尾井坂で石川県士族の島田一郎らに暗殺された政治家は誰?
解答：大久保利通

問題：中国を代表するIT企業の総称「BATH」で表される4つの会社といえば、Baidu、Alibaba、Tencentと何?
解答：Huawei

正解率：50.4


# 9.3.6 文脈内学習

In [30]:
quiz_train_dataset = load_dataset("llm-book/aio", split="train")
num_in_context_examples = 3
in_context_examples = [
    quiz_train_dataset[i] for i in range(num_in_context_examples)
]

for example in in_context_examples:
    print(f"問題：{example['question']}")
    print(f"解答：{example['answers'][0]}")

問題：「abc 〜the first〜」へようこそ!さて、ABC・・・と始まるアルファベットは、全部で何文字でしょう?
解答：26文字
問題：人気漫画『ドラえもん』の登場人物で、ジャイアンの苗字は剛田ですが、スネ夫の苗字は何でしょう?
解答：骨川
問題：格闘家ボブ・サップの出身国はどこでしょう?
解答：アメリカ


In [31]:
class InContextPromptMaker(PromptMaker):
    def __init__(self, examples: list[tuple[str, str]]):
        self._prompt = (
            "あなたには今からクイズに答えてもらいます。問題を与えますので、"
            "その解答のみを簡潔に出力してください。\n\n"
        )
        for question, answer in examples:
            self._prompt += f"問題：{question}\n解答：{answer}\n\n"

    def run(self, questions):
        prompts = [
            self._prompt + f"問題：{q}\n解答：" for q in questions
        ]
        return prompts

In [32]:
q_and_a_list = [
    (e["question"], e["answers"][0]) for e in in_context_examples
]
in_context_prompt_maker = InContextPromptMaker(q_and_a_list)
prompt = in_context_prompt_maker.run(["日本で一番高い山は何?"])
print(prompt)

['あなたには今からクイズに答えてもらいます。問題を与えますので、その解答のみを簡潔に出力してください。\n\n問題：「abc 〜the first〜」へようこそ!さて、ABC・・・と始まるアルファベットは、全部で何文字でしょう?\n解答：26文字\n\n問題：人気漫画『ドラえもん』の登場人物で、ジャイアンの苗字は剛田ですが、スネ夫の苗字は何でしょう?\n解答：骨川\n\n問題：格闘家ボブ・サップの出身国はどこでしょう?\n解答：アメリカ\n\n問題：日本で一番高い山は何?\n解答：']


In [35]:
in_context_output_answers = get_chatgpt_outputs_for_quiz(
    in_context_prompt_maker,
    quiz_dataset,
    batch_size=4,
)
in_context_accuracy = calculate_quiz_accuracy(
    in_context_output_answers,
    [e["answers"] for e in quiz_dataset],
)
print(f"正解率：{in_context_accuracy * 100}")

  0%|          | 4/1000 [00:00<03:38,  4.57it/s]

問題：映画『ウエスト・サイド物語』に登場する2つの少年グループといえば、シャーク団と何団?
解答：ジェット団

問題：氷った海に穴を開けて漁をすることから、漢字で「氷の下の魚」と書くタラ科の魚は何?
解答：鱈

問題：アメリカとロシア間で結ばれた「戦略兵器削減条約」のことをアルファベット5文字の略称で何という?
解答：START

問題：リンゴが木から落ちる様子を見て「万有引力」を発見したといわれる、17世紀のイギリスの科学者は誰?
解答：アイザック・ニュートン



  1%|          | 8/1000 [00:01<03:45,  4.40it/s]

問題：聖武天皇の時代に栄えた、東大寺正倉院や唐招提寺金堂など、中国・唐の影響を強く受けた貴族文化を何という?
解答：奈良時代

問題：アメリカ・メジャーリーグの2つのリーグといえば、ナショナルリーグと何リーグ?
解答：アメリカンリーグ

問題：池坊、草月流、小原流といえば、どんな分野の三大流派?
解答：華道

問題：英語で「取り巻く」という意味がある、前後左右から音が聞こえる立体音響を何という?
解答：サラウンド音声



  1%|          | 12/1000 [00:02<03:03,  5.39it/s]

...

問題：19世紀にバイエルン王国のルートヴィヒ1世の結婚式のお祝いとして開かれた祝賀会を起源とする、毎年秋にドイツのミュンヘンで開催される世界最大のビール祭りは何?
解答：オクトーバーフェスト

問題：薬局で働く専門の職業で、お医者さんからの指示に基づいて、患者に薬を提供したり、薬を調合したりする資格を持つ人を何師という?
解答：薬剤師

問題：エビやイカ、タコなどの血液が青く見える主な要因である、血液中に含まれる、銅を含む色素たんぱく質を何という?
解答：ヘマシアニン

問題：日本で最初の元号「大化」が始まったのは西暦何年?
解答：645年



 57%|█████▋    | 572/1000 [01:39<01:15,  5.66it/s]

問題：安納芋、宇宙センター、鉄砲伝来。これらから連想される鹿児島県の島はどこ?
解答：屋久島

問題：旅館の部屋に置かれている、衣類や手回り品などを一時的に入れておくための浅い箱を何という?
解答：箪笥

問題：ゾウを主人公とした絵本で『ぞうのエルマー』の生みの親はデビッド・マッキーですが、『ぞうのババール』の生みの親であるフランスの作家は誰?
解答：ジャン・ド・ブリュノフ

問題：もともとは、映画用のフィルムの縁にある音声を記録する部分のことで、そこから、映画やドラマのBGMや挿入曲を指すようになった言葉といえば何?
解答：サウンドトラック



 57%|█████▋    | 572/1000 [01:40<01:15,  5.70it/s]


RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo in organization org-Kx69x7PdKOLYJZcJv5CL0aM9 on tokens per min (TPM): Limit 60000, Used 59843, Requested 201. Please try again in 44ms. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}